In [1]:
import rlepose.datasets.lxd_freihand as lxd_freihand
from rlepose.utils.config import update_config
cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")

root_dir = "/home/louxd/dataset/FreiHand"
split0_train = "FreiHAND_pub_v2/training"
split1_train = "FreiHAND_pub_v2"
split2_train = "training"
split0_eval = "FreiHAND_pub_v2_eval/evaluation"
split1_eval = "FreiHAND_pub_v2_eval"
split2_eval = "evaluation"
mode_train = "train"
mode_eval = "eval"
batch_size = 32
train_dataset = lxd_freihand.Freihand_CustomDataset(root_dir, split0_train, split1_train, split2_train,
                                            cfg, mode=mode_train)

AUG ON.


In [2]:
import random
import numpy as np
import torch
import torch.multiprocessing as mp
import torch.utils.data

num_gpu = torch.cuda.device_count()


def _init_fn(worker_id):
    np.random.seed(1234)
    random.seed(1234)

train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, num_replicas=16, rank=5)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=(train_sampler is None), num_workers=16, sampler=train_sampler, worker_init_fn=_init_fn)


In [3]:
from rlepose.models import builder
device = torch.device('cuda')
def preset_model(cfg):
    model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)

    if cfg.MODEL.PRETRAINED:
        model.load_state_dict(torch.load(cfg.MODEL.PRETRAINED))
    elif cfg.MODEL.TRY_LOAD:
        pretrained_state = torch.load(cfg.MODEL.TRY_LOAD)
        model_state = model.state_dict()
        pretrained_state = {k: v for k, v in pretrained_state.items()
                            if k in model_state and v.size() == model_state[k].size()}

        model_state.update(pretrained_state)
        model.load_state_dict(model_state)
    else:
        model._initialize()

    return model

m = preset_model(cfg)
m.cuda(device)

build single from cfg.


/home/louxd/miniconda3/envs/deeplearning/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/louxd/miniconda3/envs/deeplearning/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RegressFlow(
  (preact): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size

In [4]:
criterion = builder.build_loss(cfg.LOSS).cuda()
from rlepose.utils.metrics import DataLogger, calc_accuracy, calc_coord_accuracy, evaluate_mAP

m.train()
hm_shape = cfg.DATA_PRESET.get('HEATMAP_SIZE')
depth_dim = cfg.MODEL.get('DEPTH_DIM')
output_3d = cfg.DATA_PRESET.get('OUT_3D', False)
hm_shape = (hm_shape[1], hm_shape[0], depth_dim)
grad_clip = cfg.TRAIN.get('GRAD_CLIP', False)

for i, (inps, labels) in enumerate(train_loader):
        inps = inps.cuda()

        for k, _ in labels.items():
            if k == 'type':
                continue
            
            labels[k] = labels[k].cuda(device)

        output = m(inps, labels)

        loss = criterion(output, labels)
        if cfg.TEST.get('HEATMAP2COORD') == 'heatmap':
            acc = calc_accuracy(output, labels)
        elif cfg.TEST.get('HEATMAP2COORD') == 'coord':
            acc = calc_coord_accuracy(output, labels, hm_shape, output_3d=output_3d)
        
        if i == 1:
             raise

build single from cfg.


In [9]:
torch.save(m.module.state_dict(), './exp/{}-{}/final.pth'.format(opt.exp_id, cfg.FILE_NAME))
print(f'Loading model from {opt.checkpoint}...')
# print('Loading model from {}...'.format(opt.checkpoint))
m.load_state_dict(torch.load(opt.checkpoint, map_location='cpu'), strict=True)  # 加载权重

(48, 64, None)